In [16]:
from neo4j import GraphDatabase

# Connection setup
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "16zomole"))  # Replace with your credentials

def add_ast(tx, node_id, node_type, name=None):
    """ Function to add an AST node to the Neo4j database """
    if name:
        query = (
            "CREATE (n:ASTNode {id: $node_id, type: $node_type, name: $name}) "
            "RETURN n"
        )
        result = tx.run(query, node_id=node_id, node_type=node_type, name=name)
    else:
        query = (
            "CREATE (n:ASTNode {id: $node_id, type: $node_type}) "
            "RETURN n"
        )
        result = tx.run(query, node_id=node_id, node_type=node_type)
    return result.single()[0]

def add_relationship(tx, parent_id, child_id):
    """ Function to add a relationship between two AST nodes """
    query = (
        "MATCH (a:ASTNode), (b:ASTNode) "
        "WHERE a.id = $parent_id AND b.id = $child_id "
        "CREATE (a)-[:PARENT_OF]->(b)"
    )
    tx.run(query, parent_id=parent_id, child_id=child_id)


In [17]:
def main():
    # Hardcoded AST nodes (simplified version of your structure)
    ast_nodes = {
        "file": ("File", "main"),
        "import": ("GenDecl", "import"),
        "func": ("FuncDecl", "main"),
    }

    # Relationships based on AST structure
    relationships = [
        ("file", "import"),
        ("file", "func"),
    ]

    with driver.session() as session:
        # Adding nodes
        for node_id, (node_type, name) in ast_nodes.items():
            session.write_transaction(add_ast, node_id, node_type, name)

        # Adding relationships
        for parent_id, child_id in relationships:
            session.write_transaction(add_relationship, parent_id, child_id)

        print("AST loaded into Neo4j successfully!")


In [18]:
main()

AST loaded into Neo4j successfully!


/var/folders/x2/0f74_mmn0ts5qbnsn_wp241w0000gn/T/ipykernel_5259/3396984112.py:18: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_ast, node_id, node_type, name)
/var/folders/x2/0f74_mmn0ts5qbnsn_wp241w0000gn/T/ipykernel_5259/3396984112.py:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_relationship, parent_id, child_id)


In [24]:
def get_ast_node(tx, node_id):
    """ Function to retrieve an AST node from the Neo4j database """
    query = "MATCH (n:ASTNode {id: $node_id}) RETURN n"
    result = tx.run(query, node_id=node_id)
    record = result.single()  # Get the single record if available
    return record["n"] if record else None


def get_children(tx, parent_id):
    """ Function to retrieve child nodes of a specified parent node """
    query = (
        "MATCH (p:ASTNode {id: $parent_id})-[:PARENT_OF]->(c:ASTNode) "
        "RETURN c"
    )
    result = tx.run(query, parent_id=parent_id)
    return [record["c"] for record in result]


In [25]:
session = driver.session()
node = session.execute_read(get_ast_node, "file")
if node:
    print("Retrieved Node:", node)
else:
    print("Node not found.")

        # Get children of a specific node
children = session.read_transaction(get_children, "file")
print("Children of the node:", [child for child in children])


Retrieved Node: <Node element_id='4:5e1d3220-7b75-49f7-9fdd-98681d96641a:0' labels=frozenset({'ASTNode'}) properties={'name': 'main', 'id': 'file', 'type': 'File'}>
Children of the node: [<Node element_id='4:5e1d3220-7b75-49f7-9fdd-98681d96641a:5' labels=frozenset({'ASTNode'}) properties={'name': 'main', 'id': 'func', 'type': 'FuncDecl'}>, <Node element_id='4:5e1d3220-7b75-49f7-9fdd-98681d96641a:2' labels=frozenset({'ASTNode'}) properties={'name': 'main', 'id': 'func', 'type': 'FuncDecl'}>, <Node element_id='4:5e1d3220-7b75-49f7-9fdd-98681d96641a:4' labels=frozenset({'ASTNode'}) properties={'name': 'import', 'id': 'import', 'type': 'GenDecl'}>, <Node element_id='4:5e1d3220-7b75-49f7-9fdd-98681d96641a:1' labels=frozenset({'ASTNode'}) properties={'name': 'import', 'id': 'import', 'type': 'GenDecl'}>, <Node element_id='4:5e1d3220-7b75-49f7-9fdd-98681d96641a:2' labels=frozenset({'ASTNode'}) properties={'name': 'main', 'id': 'func', 'type': 'FuncDecl'}>, <Node element_id='4:5e1d3220-7b75-49

/var/folders/x2/0f74_mmn0ts5qbnsn_wp241w0000gn/T/ipykernel_5259/1186447875.py:9: DeprecationWarning: read_transaction has been renamed to execute_read
  children = session.read_transaction(get_children, "file")


In [27]:
node = session.execute_read(get_ast_node, "file")


In [28]:
node

<Node element_id='4:5e1d3220-7b75-49f7-9fdd-98681d96641a:0' labels=frozenset({'ASTNode'}) properties={'name': 'main', 'id': 'file', 'type': 'File'}>